In [ ]:
"""
    Instruções:
        1- Download dos dados municipais do estado de Minas Gerais;
        2- Transformação dos dados de desmatamento;
        3- Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        4- Geração dos arquivos de saída no formato GeoJSON;
"""

In [8]:
import requests
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import os

# URL do dataset de municípios de Minas Gerais
url = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json'
gdf = gpd.read_file(url)

# Focos de desmatamento para agosto
file_path_ago22 = 'dados/desmatamento_ago22.gpkg'
gdf_ago22 = gpd.read_file(file_path_ago22)

# Focos de desmatamento para setembro
file_path_set22 = 'dados/desmatamento_set22.gpkg'
gdf_set22 = gpd.read_file(file_path_set22)

# Juntar os GeoDataFrames (Agosto e Setembro de 2022)
gdf_combined = pd.concat([gdf_ago22, gdf_set22], ignore_index=True)

# Projeção EPSG:31983 (desmatamento)
target_crs = CRS.from_epsg(31983)
gdf_combined = gdf_combined.to_crs(target_crs)

# Projeção EPSG:31983 (Municípios) 
target_crs = CRS.from_epsg(31983)
gdf = gdf.to_crs(target_crs)

# Adicione uma coluna com a área em km² dos municípios
gdf['area_km2'] = gdf['geometry'].area / 1e6  # Converter de m² para km²

# Diretório para salvar o GeoJSON (Municípios)
output_directory_municipios = 'dados'
output_geojson_filename_municipios = 'municipios-mg.geojson'
output_geojson_path_municipios = os.path.join(output_directory_municipios, output_geojson_filename_municipios)
gdf.to_file(output_geojson_path_municipios, driver='GeoJSON')
print(f"GeoJSON de Municípios salvo em {output_geojson_path_municipios}")

# Diretório para salvar o GeoJSON (Desmatamento)
output_directory_desmatamento = 'dados'
output_geojson_filename_desmatamento = 'focos-desmatamento-mg.geojson'
output_geojson_path_desmatamento = os.path.join(output_directory_desmatamento, output_geojson_filename_desmatamento)
gdf_combined.to_file(output_geojson_path_desmatamento, driver='GeoJSON')
print(f"GeoJSON de Desmatamento salvo em {output_geojson_path_desmatamento}")
